In [1]:
import re
import os
import yaml
import json
import pandas as pd
from google.cloud import bigquery
import google.api_core.exceptions as google_exceptions

In [2]:
config  = yaml.safe_load(open('config.yaml', 'r'))

In [3]:
query_dir = 'queries'

In [4]:
queries = {}
for query_file in (os.listdir(query_dir)):
    with open(os.path.join(query_dir, query_file), 'r') as query:
        queries[query_file] = query.read()

In [5]:
query_client = bigquery.Client()

In [6]:
def exists_table(table_reference, client):
    try:
        client.get_table(table_reference)
        return True
    except google_exceptions.NotFound:
        return False

In [7]:
if exists_table(f'{config["project_id"]}.{config["dataset"]}.expanded_sub_properties', query_client):
    request = queries['all_new_buildings_that_have_not_been_expanded.sql']
else:
    request = queries['all_new_buildings.sql']

In [8]:
real_estate_raw = query_client.query(request).to_dataframe()

In [9]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
ad_id             60 non-null int64
new_building      60 non-null bool
property_table    60 non-null object
property_attrs    60 non-null object
address           60 non-null object
dtypes: bool(1), int64(1), object(3)
memory usage: 2.0+ KB


In [10]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [11]:
real_estate_raw.head()

,new_building,property_table,property_attrs,address
ad_id,,,,
69757913,True,"{""Bolig"": [""34"", ""41"", ""43"", ""49"", ""50"", ""52"",...","{""Fellesgjeld"": ""0 kr"", ""Omkostninger"": ""10\u0...","Siggerudbråten, 2080 Eidsvoll"
88841945,True,"{""Bolig"": [""C4 01"", ""C4 09"", ""C4 17"", ""C4 24"",...","{""Areal"": ""37\u00a0-\u00a087 m\u00b2"", ""Sovero...","Lørenveien 57-63, 0585 Oslo"
90475285,True,"{""Bolig"": [""A206"", ""B205"", ""B301"", ""B403"", ""B4...","{""Omkostninger"": ""29\u00a0154 \u2013 42\u00a01...","Skaret, Kolbotn - Ormerudveien 8, 1410 Kolbotn"
96236122,True,"{""Bolig"": [""B B601"", ""B B602"", ""B B604"", ""C C6...","{""Areal"": ""70\u00a0-\u00a090 m\u00b2"", ""Sovero...","Skårersletta 50 - Hus B-C, 1473 Lørenskog"
98054029,True,"{""Bolig"": [""A-H0201"", ""A-H0203"", ""A-H0301"", ""A...","{""Omkostninger"": ""8\u00a0056 \u2013 15\u00a038...","Rostockgata, 0191 Oslo"


In [12]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [13]:
def expanand_property_tables(new_buildings):
    cols = new_buildings.iloc[0]['property_table'].keys()
    col_map = {'Bolig': 'apt_id',
               'Soverom': 'num_bedrooms',
               'Etg': 'floor',
               'P-rom': 'primary_size',
               'BRA': 'total_size',
               'Totalpris': 'price',
               'Pris': 'price'}
    property_tables = pd.DataFrame(columns=cols)
    for code, row in new_buildings['property_table'].iteritems():
        all_properties = [arr for arr in row.values()]
        to_append = dict(zip(cols, all_properties))
        frame = pd.DataFrame(to_append, columns=cols)
        frame['ad_id'] = code
        property_tables = property_tables.append(frame, ignore_index=True, sort=False)
    property_tables.rename(columns=col_map, inplace=True)
    return property_tables

In [14]:
sub_properties = expanand_property_tables(real_estate_raw[(real_estate_raw.new_building == True) & (real_estate_raw.property_table)])

In [15]:
sub_properties.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,34,2,-,89 m²,93 m²,3 211 622 kr,69757913.0
1,41,2,2.,89 m²,93 m²,3 261 622 kr,69757913.0
2,43,2,2.,89 m²,93 m²,3 308 582 kr,69757913.0
3,49,2,2.,89 m²,93 m²,3 409 162 kr,69757913.0
4,50,2,2.,67 m²,71 m²,2 908 582 kr,69757913.0


In [16]:
def get_int_from_str(string):
    try:
        concat_string = ''.join(re.findall(r'[\d.]', string))
        if concat_string != '':
            return float(concat_string)
    except Exception as e:
        print(e, string)
        pass

In [17]:
sub_properties_parsed = sub_properties.copy()
for col in ['primary_size', 'floor', 'price', 'num_bedrooms', 'total_size']:
    sub_properties_parsed[col] = [get_int_from_str(s) for s in sub_properties_parsed[col]]

expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None
expected string or bytes-like object None


In [18]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 7 columns):
apt_id          742 non-null object
num_bedrooms    733 non-null float64
floor           612 non-null float64
primary_size    691 non-null float64
total_size      742 non-null float64
price           726 non-null float64
ad_id           742 non-null float64
dtypes: float64(6), object(1)
memory usage: 40.7+ KB


In [19]:
sub_properties_parsed.head()

,apt_id,num_bedrooms,floor,primary_size,total_size,price,ad_id
0,34,2.0,NaN,89.0,93.0,3211622.0,69757913.0
1,41,2.0,2.0,89.0,93.0,3261622.0,69757913.0
2,43,2.0,2.0,89.0,93.0,3308582.0,69757913.0
3,49,2.0,2.0,89.0,93.0,3409162.0,69757913.0
4,50,2.0,2.0,67.0,71.0,2908582.0,69757913.0


In [20]:
sub_properties_parsed.drop_duplicates(inplace=True)

In [21]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 741
Data columns (total 7 columns):
apt_id          742 non-null object
num_bedrooms    733 non-null float64
floor           612 non-null float64
primary_size    691 non-null float64
total_size      742 non-null float64
price           726 non-null float64
ad_id           742 non-null float64
dtypes: float64(6), object(1)
memory usage: 46.4+ KB


In [22]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             if_exists='append')